In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
import ast
import requests

class Agent():
    def __init__(self) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.model = AutoModel.from_pretrained('bert-base-uncased')

    def words2vector(self, sentence: str):
        sentence = "This is an example sentence."
        tokens = self.tokenizer.tokenize(sentence)

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_ids = [self.tokenizer.cls_token_id] + input_ids + [self.tokenizer.sep_token_id]

        max_length = 64
        input_ids = input_ids + [self.tokenizer.pad_token_id] * (max_length - len(input_ids))
        input_tensor = torch.tensor([input_ids])

        with torch.no_grad():
            outputs = self.model(input_tensor)
            embedding = outputs[0][:, 0, :].numpy()

        return embedding[0]
    
model = Agent()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
qv = np.array(model.words2vector('Active ingredient'))
qv = np.array2string(qv, separator=',', formatter={'float_kind': lambda x: "%.2f" % x})
qv

'[-0.64,0.22,0.31,0.69,-0.21,-0.58,0.60,-0.13,0.31,-0.11,-0.37,-0.45,-0.33,\n 0.67,0.60,0.67,-0.18,0.52,0.08,-0.30,0.42,0.47,0.93,0.11,-0.05,-0.07,\n -0.05,-1.07,-0.51,-0.03,-0.48,0.53,0.07,-0.07,0.41,-0.47,-0.67,-0.21,0.41,\n 0.21,0.03,-0.65,0.83,0.07,-0.25,-0.58,-3.69,0.28,0.01,-0.50,-0.21,-0.29,\n 0.54,0.20,0.18,0.82,0.03,-0.56,1.60,0.22,0.11,0.54,-0.72,0.02,-0.07,0.66,\n 0.21,-0.10,-0.67,0.28,0.08,0.46,0.54,0.12,-0.08,-0.36,-0.38,1.25,-0.58,\n -0.26,0.24,-0.32,0.02,-0.85,0.55,0.26,0.58,-0.52,-0.63,0.20,0.19,0.08,\n 0.43,0.35,0.11,-0.21,0.13,0.13,-0.24,-0.01,-0.61,1.07,0.40,-0.35,-0.11,\n 0.05,-1.37,0.06,-0.13,0.23,0.19,0.51,-0.71,0.04,0.51,0.25,0.37,-0.38,0.68,\n -0.43,0.49,-0.03,0.18,0.07,-0.04,0.04,0.05,-1.54,0.37,0.90,0.03,0.34,\n -0.40,0.11,-0.48,-0.64,-0.00,0.16,-0.10,0.08,-1.20,-0.61,-0.48,-0.75,0.26,\n -0.02,0.22,0.58,-0.54,-0.33,-0.14,0.02,-1.13,-0.83,-0.33,-0.78,0.90,-0.05,\n -0.20,0.89,0.26,0.65,0.16,0.51,-0.19,0.04,0.71,0.43,-0.10,-0.24,0.01,0.01,\n 0.71,-0.74,-0.01,0.51

In [28]:
URL = 'http://localhost:8983/solr/ir_updated/select?indent=true&q.op=OR&&q={!knn f=feature_vector topK=10}' + qv
r = requests.get(url = URL)

In [29]:
with open("response.txt", "wb") as f:
    f.write(r.content)
r

<Response [200]>